#### Studying the effect of Pruning Factor on Iteraitve Model Pruning for Proposed architectures using formula 

N_new = floor(N_old * (1 - Pruning_Factor))

Where:
- N_new: New number of filters/dense units after pruning
- N_old: Old number of filters/dense units before pruning
- Pruning_Factor: Pruning factor applied, expressed as a decimal


In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPooling2D,
    Flatten,
    BatchNormalization,
    Dropout,
    concatenate,
    ZeroPadding2D
)


In [ ]:
import pandas as pd

def count_model_parameters(model):
    """
    Count the total number of trainable parameters in a Keras model.

    Parameters:
    ----------
    model : tf.keras.Model
        The Keras model for which parameters are to be counted.

    Returns:
    -------
    int
        Total number of trainable parameters in the model.
    """
    return model.count_params()

def reduce_filters(filters, factor):
    """
    Reduce the number of filters in convolutional layers by a specified factor.

    Parameters:
    ----------
    filters : list
        List of integers representing the number of filters in convolutional layers.
    factor : float
        Reduction factor for the number of filters.

    Returns:
    -------
    list
        Updated list of reduced filters.
    """
    return [max(1, int(f * factor)) for f in filters]

def reduce_dense_units(dense_units, factor):
    """
    Reduce the number of units in dense layers by a specified factor.

    Parameters:
    ----------
    dense_units : list
        List of integers representing the number of units in dense layers.
    factor : float
        Reduction factor for the number of units.

    Returns:
    -------
    list
        Updated list of reduced dense units.
    """
    return [max(1, int(u * factor)) for u in dense_units]

def compute_parameters_for_architecture(build_model_fn, initial_filters, initial_dense_units, pruning_factor, num_reductions, output_csv, image_sizes=None, save_for_image_sizes=False, **kwargs):
    """
    Compute the number of parameters for different configurations of a given model architecture 
    and save the results to CSV file(s).
    
    This is stored in a csv file which can be analyse to study effects of change in model size and pruning in number of filters and 
    dense units when 1. Alll convolution layers and dense units of image model are reduce, if only one conv layer along with dense units of model is reduced etc.

    Parameters:
    ----------
    build_model_fn : function
        Function to build the Keras model with specific parameters.
    initial_filters : list
        Initial values for the number of filters in convolutional layers.
    initial_dense_units : list
        Initial values for the number of units in dense layers.
    pruning_factor : float
        Factor by which to reduce filters and dense units in each iteration.
    num_reductions : int
        Number of reductions to apply.
    output_csv : str
        File path to save the CSV file(s) with computed parameters.
    image_sizes : list of tuples, optional
        List of tuples specifying different input image sizes (height, width, channels). Default is None.
    save_for_image_sizes : bool, optional
        Whether to save CSVs for each image size configuration. Default is False.
    **kwargs : dict
        Additional keyword arguments for future extensibility.

    Returns:
    -------
    None
    """
    if image_sizes is None:
        image_sizes = []

    configs = []
    original_filters = initial_filters.copy()
    original_dense_units = initial_dense_units.copy()

    if image_sizes:
        for image_size in image_sizes:
            # Build model with the given image size
            model = build_model_fn(original_filters, original_dense_units, input_shape=image_size)
            num_params = count_model_parameters(model)
            configs.append((image_size, original_filters, original_dense_units, num_params))
            if save_for_image_sizes:
                df = pd.DataFrame(configs, columns=['Image Size', 'Filters', 'Dense Units', 'Num Parameters'])
                df.to_csv(f"{output_csv}_image_size_pruning", index=False)

    # Build model with default input shape
    model = build_model_fn(original_filters, original_dense_units)
    num_params = count_model_parameters(model)
    configs.append(('Default Input Shape', original_filters, original_dense_units, num_params))

    for i in range(num_reductions + 1):
        # Reduce all convolutional layers and dense layers
        current_filters = reduce_filters(original_filters, pruning_factor**i)
        current_dense_units = reduce_dense_units(original_dense_units, pruning_factor**i)
        model = build_model_fn(current_filters, current_dense_units)
        num_params = count_model_parameters(model)
        configs.append((current_filters, current_dense_units, num_params, f'All Layers (Reduction {i})'))

    # Reduce all conv layer, 1 conv layer, 2 conv layer or 3 conv layer...
    for i in range(1, len(original_filters) + 1):
        current_filters = original_filters.copy()
        for j in range(num_reductions + 1):
            temp_filters = current_filters.copy()
            temp_filters[:i] = reduce_filters(original_filters[:i], pruning_factor**j)
            temp_dense_units = reduce_dense_units(original_dense_units, pruning_factor**j)
            model = build_model_fn(temp_filters, temp_dense_units)
            num_params = count_model_parameters(model)
            configs.append((temp_filters, temp_dense_units, num_params, f'First {i} Layer(s) (Reduction {j})'))

    # Save the results to a CSV file
    df = pd.DataFrame(configs, columns=['Filters', 'Dense Units', 'Num Parameters', 'Configuration'])
    df.to_csv(f"{output_csv}_parameter_reduction", index=False)

    print(df)


In [ ]:


def build_example_net(num_filters, num_dense_units, input_shape=(200, 100, 3)):

    text_model = Sequential()
    text_model.add(Dense(4, input_dim=3, activation="relu"))
    text_model.add(BatchNormalization())
    text_model.add(Dropout(0.25))

    text_model.add(Dense(6, input_dim=3, activation="relu"))
    text_model.add(BatchNormalization())

    text_model.add(Dense(2, input_dim=3, activation="relu"))
    text_model.add(BatchNormalization())
    text_model.add(Dropout(0.25))

    text_model.add(Dense(units=1, activation="relu"))  # Adjust units for 7 classes
    model = Sequential()
    model.add(Conv2D(num_filters[0], (5, 5), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(num_filters[1], (5, 5), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(num_dense_units[0], activation='relu'))
    model.add(Dense(num_dense_units[1], activation='relu'))
    combined_output = concatenate([text_model.output, model.output])

    x = Dense(8, activation="relu")(combined_output)
    x = Dense(4, activation="softmax")(x)

    model = Model([text_model.input, model.input], x)

    return model

# EXAMPLE USAGE
# This can be extended to any architecture given build model function is defined as per requirement and analysis just as it is defined above in build_example_net
initial_filters = [6, 16]
initial_dense_units = [84, 120]
pruning_factor = 0.5
num_reductions = 5
output_csv = f'Lenet_{pruning_factor}_parameters.csv'
image_sizes = [(200, 100, 3), (400, 200, 3)]

compute_parameters_for_architecture(build_example_net, initial_filters, initial_dense_units, pruning_factor, num_reductions, output_csv, image_sizes=image_sizes, save_for_image_sizes=True)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_parameters_from_csv(csv_file, title='Parameter vs Image Size', parameter_fontsize=12, title_fontsize=14):
    """
    Plot number of learnable parameters against Image Size from a CSV file generated after 'compute_parameters_for_architecture' Func .

    Parameters:
    - csv_file (str): Path to the CSV file containing 'Image_Size' and 'Parameter' columns.
    - title (str, optional): Title of the plot. Default is 'Parameter vs Image Size'.
    - parameter_fontsize (int, optional): Font size for parameters. Default is 12.
    - title_fontsize (int, optional): Font size for title. Default is 14.
    """
    # Read CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)

    # Sort DataFrame by 'Num Parameters'
    df_sorted = df.sort_values(by='Num Parameters',ascending=False)

    # Extract sorted columns
    image_sizes = df_sorted['Image Size']
    parameters = df_sorted['Num Parameters']

    # Plotting
    fig, ax = plt.subplots(figsize=(7, 4.5))  # Adjust figsize as needed

    # Plot parameters against image sizes
    ax.plot(image_sizes, parameters, marker='o', linestyle='-', color='b', markersize=5)

    # Set title and adjust fontsize
    ax.set_title(title, fontsize=title_fontsize,pad=20)

    # Labeling axes
    ax.set_xlabel('Image Size', fontsize=12)
    ax.set_ylabel('Parameter', fontsize=parameter_fontsize)

    # Set x-axis ticks (optional depending on data)
     # Set x-axis and y-axis tick label font sizes
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    # Rotate x-axis labels by 45 degrees
    plt.xticks(rotation=45)
    # Display grid
    ax.grid(True)

    # Automatically adjust layout
    fig.tight_layout()

    # Show plot
    plt.show()

# Example usage:
# Change Input File as per requirement and modify parameters like title, fontsize 
csv_file = 'Custom_Cnn_2_input_shape_parameters.csv'  # Replace with your CSV file path

plot_parameters_from_csv(csv_file, title='Custom Neural Network - 2 : #Parameters vs Image Size', parameter_fontsize=12, title_fontsize=16)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

def plot_parameters_from_csv(csv_file, title='Num Parameters vs Image Size', parameter_fontsize=12, title_fontsize=14,
                             xtick_fontsize=10, ytick_fontsize=10, legend_fontsize=10):
    """
    Plot parameters against Reduction Number from a CSV file for each unique Configuration and determine trend type.

    Parameters:
    - csv_file (str): Path to the CSV file containing 'Reduction Number', 'Configuration', and 'Num Parameters' columns.
    - title (str, optional): Title of the plot. Default is 'Num Parameters vs Reduction Number'.
    - parameter_fontsize (int, optional): Font size for parameters. Default is 12.
    - title_fontsize (int, optional): Font size for title. Default is 14.
    - xtick_fontsize (int, optional): Font size for x-axis tick labels. Default is 10.
    - ytick_fontsize (int, optional): Font size for y-axis tick labels. Default is 10.
    - legend_fontsize (int, optional): Font size for legend. Default is 10.
    """
    # Read CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)

    fig, ax = plt.subplots(figsize=(8, 5))  # Adjust figsize as needed

    # Iterate over each unique configuration
    Label = ["All Conv Layers", "1 Conv Layer", "2 Conv Layers", "3 Conv Layers"]
    colors = ['b', 'g', 'r', 'c']
    trend_types = []

    for j, label in enumerate(Label):
        # Filter data for current configuration
        df_config = df[df['Configuration'] == label]

        # Extract Reduction Number and Num Parameters
        reduction_numbers = df_config['Reduction Number']
        num_parameters = df_config['Num Parameters']

        # Plot parameters against Reduction Number
        ax.plot(reduction_numbers, num_parameters, marker='o', linestyle='-', markersize=5, label=f'{label}', color=colors[j])

        # Perform linear regression to determine trend
        slope, intercept, r_value, p_value, std_err = linregress(reduction_numbers, num_parameters)

        # Determine trend type based on R-squared value
        if r_value ** 2 > 0.95:
            trend_type = 'Exponential'
        elif r_value ** 2 > 0.85:
            trend_type = 'Polynomial'
        else:
            trend_type = 'Linear'

        trend_types.append(trend_type)

    # Set title and adjust fontsize
    ax.set_title(title, fontsize=title_fontsize, pad=20)

    # Labeling axes
    ax.set_xlabel('Reduction Number', fontsize=parameter_fontsize)
    ax.set_ylabel('Num Parameters', fontsize=parameter_fontsize)

    # Set x-axis and y-axis tick label font sizes
    ax.tick_params(axis='x', labelsize=xtick_fontsize)
    ax.tick_params(axis='y', labelsize=ytick_fontsize)

    # Display grid
    ax.grid(True)

    # Add legend
    ax.legend(fontsize=legend_fontsize)

    # Automatically adjust layout
    fig.tight_layout()

    # Show plot
    plt.show()

    # Print trend types
    for j, label in enumerate(Label):
        print(f"Trend type for {label}: {trend_types[j]}")

# Example usage:
csv_file = 'Custom_Cnn_1_0.66_parameters.csv'  # Replace with your CSV file path

plot_parameters_from_csv(csv_file, title='Custom Neural Network - Num Parameters vs Reduction Number',
                         parameter_fontsize=12, title_fontsize=16,
                         xtick_fontsize=12, ytick_fontsize=12, legend_fontsize=12)
